In [1]:
# requests 라이브러리 설치여부 확인
!pip install requests

In [2]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip install beautifulsoup4

In [3]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [10]:
import os
import json
import urllib.request
import requests
from datetime import datetime
import time
from bs4 import BeautifulSoup

# 요청 헤더 (사람처럼 보이게 설정)
REQ_HEADER = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

def get_climate_keywords():
    """기후 관련 고정 키워드 리스트 반환"""
    keywords = [
        "날씨", "울산날씨", "강풍 경보", "부산날씨", "날씨예보", 
        "내일 날씨", "일기예보", "대구 날씨", "天気", "wind warning"
    ]
    print(f"🌡️ 기후 관련 키워드 {len(keywords)}개를 설정했습니다.")
    return keywords

def collect_portal_news(portal_name, keyword):
    encoded_kw = urllib.parse.quote(keyword)
    
    if portal_name == "Daum":
        url = f"https://search.daum.net/search?w=news&q={encoded_kw}"
    elif portal_name == "Nate":
        url = f"https://search.nate.com/search?q={encoded_kw}&thr=ct"
    else:
        return []

    news_list = []

    try:
        res = requests.get(url, headers=REQ_HEADER, timeout=5)
        if not res.ok:
            return news_list

        soup = BeautifulSoup(res.text, 'html.parser')

        if portal_name == "Daum":
            a_tags = soup.select("div.item-title a")
        else:  # Nate
            a_tags = soup.select("div.news_wrap a")

        seen_titles = set()

        for a_tag in a_tags:
            title = a_tag.get_text(strip=True)
            link = a_tag.get("href", "")

            if not link or len(title) < 5:
                continue

            if link.startswith("//"):
                link = "https:" + link

            if title not in seen_titles:
                news_list.append({"title": title, "link": link})
                seen_titles.add(title)

            break  # 포털당 1개
    except Exception as e:
        print(f"❌ {portal_name} 수집 중 에러: {e}")

    return news_list

if __name__ == "__main__":
    now = datetime.now()
    print(f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] 기후 데이터 통합 수집 시작")
    
    # 1. 키워드 설정
    kws = get_climate_keywords()
    
    if kws:
        print("📰 포털(Daum/Nate) 뉴스 통합 수집 진행 중...")
        integrated_data = []
        
        for i, kw in enumerate(kws):
            # 진행 상태 출력
            print(f" 진행 중: ({i+1}/{len(kws)}) {kw}" + " " * 15, end='\r')
            
            # 다음 및 네이트 뉴스 크롤링 (네이버 제거됨)
            daum_news = collect_portal_news("Daum", kw)
            nate_news = collect_portal_news("Nate", kw)
            
            integrated_data.append({
                "category": "기후",
                "id": i + 1,
                "keyword": kw,
                "news": {
                    "daum": daum_news,
                    "nate": nate_news
                }
            })
            # 차단 방지를 위한 약간의 휴식
            time.sleep(0.7)

        # 2. 최종 결과 저장
        output = {
            "target_keywords": kws,
            "meta_info": {
                "source": "daum_news & nate_news",
                "category": "climate",
                "collected_at": now.isoformat(),
                "count": len(kws)
            },
            "data": integrated_data
        }
        
        file_name = 'climate_news_only.json'
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(output, f, ensure_ascii=False, indent=4)
        
        print(f"\n✨ 완료: {len(kws)}개 키워드 데이터를 {file_name}에 저장했습니다.")
    else:
        print("키워드가 정의되지 않아 중단합니다.")

[2026-02-25 09:42:07] 기후 데이터 통합 수집 시작
🌡️ 기후 관련 키워드 10개를 설정했습니다.
📰 포털(Daum/Nate) 뉴스 통합 수집 진행 중...
 진행 중: (10/10) wind warning               
✨ 완료: 10개 키워드 데이터를 climate_news_only.json에 저장했습니다.


In [11]:
!pip install feedparser

  Using cached feedparser-6.0.12-py3-none-any.whl.metadata (2.7 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached feedparser-6.0.12-py3-none-any.whl (81 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [feedparser]


In [17]:
!pip install apscheduler

## Google

In [24]:
import requests
import feedparser
import json
import os
import time
import urllib.parse

# =========================
# Notebook 설정
# =========================
KEYWORDS = ["강풍 경보", "날씨예보", "대구 날씨", "울산날씨", "일기예보", "오늘의 날씨"]
CATEGORY = "기후"
COUNTRY = "ko"  # 현재는 ko 고정 사용
JSON_FILE = "climate_news.json"

REQ_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}


# =========================
# JSON 저장 함수 (지정 포맷 + 기사 수 포함)
# =========================
def save_news(keyword, article, total_count):
    """
    단일 JSON 파일(JSON_FILE)에 키워드별로 누적 저장.
    - 기존 포맷 유지
    - total_count: 해당 키워드로 Google News RSS가 반환한 entries 개수
    """
    if os.path.exists(JSON_FILE):
        with open(JSON_FILE, "r", encoding="utf-8") as f:
            content = json.load(f)
    else:
        content = {"data": []}

    # 이미 있는 키워드인지 확인
    for item in content["data"]:
        if item["keyword"] == keyword:
            # 기사 1건 추가
            item["news"]["google"].append(article)
            # 최신 total_count 갱신
            item["total_count"] = total_count
            break
    else:
        # 신규 키워드
        content["data"].append({
            "category": CATEGORY,
            "id": len(content["data"]) + 1,
            "keyword": keyword,
            "total_count": total_count,
            "news": {
                "google": [article]
            }
        })

    with open(JSON_FILE, "w", encoding="utf-8") as f:
        json.dump(content, f, ensure_ascii=False, indent=4)


# =========================
# Google News 수집 함수 (최신 1건 + 총 기사 수)
# =========================
def collect_google_news(keyword):
    print(f"▶ Google News 수집: {keyword}")

    # 키워드 URL 인코딩 (한글/공백 안전)
    encoded_kw = urllib.parse.quote(keyword)
    url = f"https://news.google.com/rss/search?q={encoded_kw}+when:1d&hl=ko&gl=KR&ceid=KR:ko"

    try:
        res = requests.get(url, headers=REQ_HEADERS, timeout=10)

        if res.status_code != 200:
            print(f"❌ 요청 실패 (status={res.status_code})")
            # 실패 시에도 count=0으로 저장하고 싶으면 아래 주석 해제
            # save_news(keyword, {"title": None, "link": None}, 0)
            return

        feed = feedparser.parse(res.text)
        total_count = len(feed.entries)

        if total_count == 0:
            print(f"⚠️ 결과 없음 (기사 수: {total_count}개)")
            # 결과 없음도 기록하고 싶으면 아래처럼 저장
            save_news(keyword, {"title": None, "link": None}, total_count)
            return

        entry = feed.entries[0]

        article = {
            "title": entry.get("title"),
            "link": entry.get("link")
        }

        save_news(keyword, article, total_count)
        print(f"✔ 저장 완료: {keyword} (기사 수: {total_count}개)")

    except Exception as e:
        print("❌ 오류:", e)
        # 오류도 기록하고 싶으면 아래 주석 해제
        # save_news(keyword, {"title": None, "link": None}, 0)


# =========================
# 실행
# =========================
for kw in KEYWORDS:
    collect_google_news(kw)
    time.sleep(0.5)

print(f"\n✅ 완료: 결과가 '{JSON_FILE}'에 저장되었습니다.")

▶ Google News 수집: 강풍 경보
✔ 저장 완료: 강풍 경보 (기사 수: 12개)
▶ Google News 수집: 날씨예보
✔ 저장 완료: 날씨예보 (기사 수: 100개)
▶ Google News 수집: 대구 날씨
✔ 저장 완료: 대구 날씨 (기사 수: 40개)
▶ Google News 수집: 울산날씨
✔ 저장 완료: 울산날씨 (기사 수: 35개)
▶ Google News 수집: 일기예보
✔ 저장 완료: 일기예보 (기사 수: 27개)
▶ Google News 수집: 오늘의 날씨
✔ 저장 완료: 오늘의 날씨 (기사 수: 100개)

✅ 완료: 결과가 'climate_news.json'에 저장되었습니다.


# Daum

In [26]:
import os
import json
import time
import requests
import urllib.parse
from datetime import datetime
from bs4 import BeautifulSoup

# 요청 헤더 (사람처럼 보이게 설정)
REQ_HEADER = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
}

def get_climate_keywords():
    """기후 관련 고정 키워드 리스트 반환"""
    keywords = [
        "날씨", "울산날씨", "강풍 경보", "부산날씨", "날씨예보",
        "내일 날씨", "일기예보", "대구 날씨"
    ]
    print(f"🌡️ 기후 관련 키워드 {len(keywords)}개를 설정했습니다.")
    return keywords

def collect_daum_news(keyword, max_items=1):
    """
    Daum 뉴스 검색에서 기사 추출
    - news_list: 기사 리스트 (최대 max_items개)
    - total_count: 현재 검색 결과 페이지에서 파싱된 '기사 링크 후보' 개수
    """
    encoded_kw = urllib.parse.quote(keyword)
    url = f"https://search.daum.net/search?w=news&q={encoded_kw}"

    news_list = []
    total_count = 0

    try:
        res = requests.get(url, headers=REQ_HEADER, timeout=5)
        if not res.ok:
            return news_list, total_count

        soup = BeautifulSoup(res.text, "html.parser")

        # 다음 뉴스 검색 결과: 보통 div.item-title a 형태
        a_tags = soup.select("div.item-title a")

        seen = set()
        candidates = []

        for a in a_tags:
            title = a.get_text(strip=True)
            link = a.get("href", "")

            # 기본 필터
            if not link or link == "#" or len(title) < 5:
                continue

            if link.startswith("//"):
                link = "https:" + link

            # 중복 제거 (title 기준)
            if title in seen:
                continue

            seen.add(title)
            candidates.append({"title": title, "link": link})

        # ✅ 기사 수(=현재 페이지에서 파싱된 후보 기사 수)
        total_count = len(candidates)

        # ✅ 실제 저장할 기사는 max_items개만
        news_list = candidates[:max_items]

    except Exception as e:
        print(f"❌ Daum 수집 중 에러: {e}")

    return news_list, total_count


if __name__ == "__main__":
    now = datetime.now()
    print(f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] Daum 기후 뉴스 수집 시작")

    # 1) 키워드 설정
    kws = get_climate_keywords()

    if not kws:
        print("키워드가 정의되지 않아 중단합니다.")
        raise SystemExit

    print("📰 다음(Daum) 뉴스 수집 진행 중...")
    integrated_data = []

    for i, kw in enumerate(kws):
        print(f" 진행 중: ({i+1}/{len(kws)}) {kw}" + " " * 15, end="\r")

        daum_news, total_count = collect_daum_news(kw, max_items=1)

        integrated_data.append({
            "category": "기후",
            "id": i + 1,
            "keyword": kw,
            "total_count": total_count,   # ✅ 기사 수 저장
            "news": {
                "daum": daum_news
            }
        })

        time.sleep(0.7)

    # 2) 최종 결과 저장
    output = {
        "target_keywords": kws,
        "meta_info": {
            "source": "daum_news",
            "category": "climate",
            "collected_at": now.isoformat(),
            "count": len(kws)
        },
        "data": integrated_data
    }

    file_name = "climate_daum_news_only.json"
    with open(file_name, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=4)

    print(f"\n✨ 완료: {len(kws)}개 키워드 데이터를 {file_name}에 저장했습니다.")

[2026-02-25 11:25:24] Daum 기후 뉴스 수집 시작
🌡️ 기후 관련 키워드 8개를 설정했습니다.
📰 다음(Daum) 뉴스 수집 진행 중...
 진행 중: (8/8) 대구 날씨               
✨ 완료: 8개 키워드 데이터를 climate_daum_news_only.json에 저장했습니다.


# Naver

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import time
import re
import json
from datetime import datetime, timezone, timedelta

# =========================
# 설정
# =========================
KEYWORDS = ["날씨", "울산날씨", "강풍 경보", "부산날씨", "날씨예보", "내일 날씨", "일기예보", "대구 날씨"]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en;q=0.8",
    "Referer": "https://www.naver.com/",
}

KST = timezone(timedelta(hours=9))


# =========================
# 유틸: 발행일로 보이는 텍스트 판별
# =========================
def looks_like_time_or_date(txt: str) -> bool:
    if not txt:
        return False
    t = txt.strip()
    if re.search(r"\d+\s*(분|시간|일|주|개월|년)\s*전", t):
        return True
    if re.search(r"\d{4}\.\d{2}\.\d{2}\.?", t):
        return True
    return False


# =========================
# 발행일 정규화: 'YYYY-MM-DD HH:MM' (KST)
# =========================
def normalize_published(published_raw: str):
    if not published_raw:
        return None

    s = published_raw.strip()

    # 1) ISO8601 (예: 2026-02-25T01:07:20+09:00)
    try:
        dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=KST)
        dt = dt.astimezone(KST)
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception:
        pass

    # 2) 날짜형 (예: 2026.02.25 or 2026.02.25.)
    m = re.search(r"(\d{4})\.(\d{2})\.(\d{2})", s)
    if m:
        y, mo, d = map(int, m.groups())
        dt = datetime(y, mo, d, 0, 0, tzinfo=KST)
        return dt.strftime("%Y-%m-%d %H:%M")

    # 3) 상대시간 (예: 3시간 전)
    m = re.search(r"(\d+)\s*(분|시간|일|주|개월|년)\s*전", s)
    if m:
        n = int(m.group(1))
        unit = m.group(2)
        now = datetime.now(KST)

        if unit == "분":
            dt = now - timedelta(minutes=n)
        elif unit == "시간":
            dt = now - timedelta(hours=n)
        elif unit == "일":
            dt = now - timedelta(days=n)
        elif unit == "주":
            dt = now - timedelta(weeks=n)
        elif unit == "개월":
            dt = now - timedelta(days=30 * n)   # 근사
        elif unit == "년":
            dt = now - timedelta(days=365 * n)  # 근사
        else:
            return None

        return dt.strftime("%Y-%m-%d %H:%M")

    return None


# =========================
# 기사 페이지에서 발행일 추출 보조: JSON-LD
# =========================
def parse_published_from_jsonld(soup: BeautifulSoup):
    for script in soup.select('script[type="application/ld+json"]'):
        try:
            txt = script.get_text(strip=True)
            if not txt:
                continue
            data = json.loads(txt)
            candidates = data if isinstance(data, list) else [data]
            for obj in candidates:
                if isinstance(obj, dict):
                    dp = obj.get("datePublished") or obj.get("dateCreated") or obj.get("dateModified")
                    if dp:
                        return dp
        except Exception:
            continue
    return None


# =========================
# 기사 페이지에서 발행일 추출 보조: meta
# =========================
def parse_published_from_meta(soup: BeautifulSoup):
    meta_props = [
        ("property", "article:published_time"),
        ("property", "og:article:published_time"),
        ("name", "article:published_time"),
        ("name", "pubdate"),
        ("name", "date"),
    ]
    for attr, key in meta_props:
        tag = soup.find("meta", attrs={attr: key})
        if tag and tag.get("content"):
            return tag["content"].strip()
    return None


# =========================
# 원문에서 발행일 추출
# =========================
def fetch_published_from_article(link: str):
    try:
        res = requests.get(link, headers=HEADERS, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        pub = parse_published_from_meta(soup)
        if pub:
            return pub

        pub = parse_published_from_jsonld(soup)
        if pub:
            return pub

        # 최후: 텍스트에서 날짜 패턴
        text = soup.get_text(" ", strip=True)
        m = re.search(r"\d{4}\.\d{2}\.\d{2}\.?\s*\d{1,2}:\d{2}", text)
        if m:
            return m.group(0)

        m = re.search(r"\d{4}\.\d{2}\.\d{2}\.?", text)
        if m:
            return m.group(0)

        return None
    except Exception:
        return None


# =========================
# 네이버 뉴스 검색: 키워드별 1개 추출
# =========================
def fetch_one_naver_news(keyword: str):
    q = urllib.parse.quote(keyword)
    url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={q}"

    res = requests.get(url, headers=HEADERS, timeout=10)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    # 제목/링크 (작동하던 방식 우선)
    a = soup.select_one('a[data-heatmap-target=".tit"]')
    if not a:
        a = soup.select_one("a.news_tit")
    if not a:
        return None

    title = a.get_text(strip=True)
    link = a.get("href")

    # 1차: 검색 결과 카드에서 발행일 시도
    published = None
    card = a.find_parent("div", class_="news_area")
    if not card:
        card = a.find_parent(["div", "li"])

    if card:
        infos = card.select("div.info_group span.info, span.info")
        for sp in infos:
            txt = sp.get_text(strip=True)
            if looks_like_time_or_date(txt):
                published = txt
                break

    # 2차: 원문에서 발행일 보강
    if published is None and link:
        published = fetch_published_from_article(link)

    return {
        "keyword": keyword,
        "title": title,
        "published_raw": published,
        "published": normalize_published(published),
        "link": link
    }


# =========================
# 실행
# =========================
# rows에서 published_raw 키 제거 (JSON에 안 나오게)
rows_no_raw = []
for item in rows:
    if isinstance(item, dict):
        new_item = dict(item)
        new_item.pop("published_raw", None)
        rows_no_raw.append(new_item)
    else:
        rows_no_raw.append(item)
        
rows = []
for kw in KEYWORDS:
    try:
        item = fetch_one_naver_news(kw)
        if item is None:
            rows.append({"keyword": kw, "title": None, "published": None, "published_raw": None, "link": None})
        else:
            rows.append(item)
    except Exception as e:
        rows.append({"keyword": kw, "title": None, "published": None, "published_raw": None, "link": None, "error": str(e)})

    time.sleep(0.5)  # 원문까지 들어가므로 딜레이 권장

df = pd.DataFrame(rows, columns=["keyword", "title", "published", "link"])
df

# =========================
# JSON 저장 (추가)
# =========================
OUT_FILE = "naver_news_climate.json"

output = {
    "target_keywords": KEYWORDS,
    "meta_info": {
        "source": "naver_news_search",
        "category": "climate",
        "collected_at": datetime.now().isoformat(),
        "count": len(KEYWORDS),
        "note": "1 article per keyword (title/link from Naver search; published normalized when possible)"
    },
    "data": rows_no_raw
}

with open(OUT_FILE, "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

print(f"✅ JSON 저장 완료: {OUT_FILE}")

✅ JSON 저장 완료: naver_news_climate.json
